In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm

import itertools

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
df = pd.read_csv('../../data/raw/NFL_wk_by_wk.csv', names=['WEEK_NUM','GAME_DAY_OF_WEEK','GAME_DATE','GAMETIME','WINNER','GAME_LOCATION','LOSER','BOXSCORE_LINK','PTS_WINNER','PTS_LOSER','YARDS_WINNER','TURNOVERS_WINNER','YARDS_LOSER','TURNOVERS_LOSER','YEAR'], index_col=False)

## In the data pull the header shows up multiple times
df = df[((df['WEEK_NUM']!='Week')&(df['WEEK_NUM']!='NULL_VALUE'))]

## Replace old team names with the newest version
df['WINNER'].replace({'St. Louis Rams'           : 'Los Angeles Rams',
                      'San Diego Chargers'       : 'Los Angeles Chargers',
                      'Houston Oilers'           : 'Tennessee Titans',
                      'Tennessee Oilers'         : 'Tennessee Titans',
                      'Los Angeles Raiders'      : 'Las Vegas Raiders',
                      'Oakland Raiders'          : 'Las Vegas Raiders',
                      'Phoenix Cardinals'        : 'Arizona Cardinals',
                      'Washington Redskins'      : 'Washington Commanders',
                      'Washington Football Team' : 'Washington Commanders'},inplace=True)
df['LOSER'].replace({'St. Louis Rams'            : 'Los Angeles Rams',
                      'San Diego Chargers'       : 'Los Angeles Chargers',
                      'Houston Oilers'           : 'Tennessee Titans',
                      'Tennessee Oilers'         : 'Tennessee Titans',
                      'Los Angeles Raiders'      : 'Las Vegas Raiders',
                      'Oakland Raiders'          : 'Las Vegas Raiders',
                      'Phoenix Cardinals'        : 'Arizona Cardinals',
                      'Washington Redskins'      : 'Washington Commanders',
                      'Washington Football Team' : 'Washington Commanders'},inplace=True)

## Data pull error, easier to fix here
df['YEAR'].replace({"99-100":'99-00',
                    '00-1':'00-01',
                    '01-2':'01-02',
                    '02-3':'02-03',
                    '03-4':'03-04',
                    '04-5':'04-05',
                    '05-6':'05-06',
                    '06-7':'06-07',
                    '07-8':'07-08',
                    '08-9':'08-09'}, inplace=True)

## Game Date to datetime
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

## Create sorted year list like '91-92' to use as categorical ordering list
first_year = sorted(df['GAME_DATE'].dt.year.unique())[0]
last_year  = sorted(df['GAME_DATE'].dt.year.unique())[-1]
sorted_years = [f"{str(yr)[-2:]}"+"-"+f"{str(yr+1)[-2:]}" for yr in range(int(first_year), int(last_year)+1) ]
increase_in_number_of_weeks_in_season = sorted_years.index('21-22')

## Change playoff week names to numbers
## prior to '21-22' season
df.loc[df['YEAR'].isin(sorted_years[:increase_in_number_of_weeks_in_season]),:] = df.loc[df['YEAR'].isin(sorted_years[:increase_in_number_of_weeks_in_season]),:].replace({'WildCard'  : '18',
                                                                                                                                                                           'Division'  : '19',
                                                                                                                                                                           'ConfChamp' : '20',
                                                                                                                                                                           'SuperBowl' : '21'})
## after to '21-22' season
df.loc[df['YEAR'].isin(sorted_years[increase_in_number_of_weeks_in_season:]),:] = df.loc[df['YEAR'].isin(sorted_years[increase_in_number_of_weeks_in_season:]),:].replace({'WildCard'  : '19',
                                                                                                                                                                           'Division'  : '20',
                                                                                                                                                                           'ConfChamp' : '21',
                                                                                                                                                                           'SuperBowl' : '22'})

## These data points are games were teams tied, the data didn't pull because of a "strong" tag
df.loc[df['PTS_WINNER']=='NULL_VALUE','PTS_WINNER'] = df.loc[df['PTS_WINNER']=='NULL_VALUE','PTS_LOSER']

## Change values to be numeric
df['WEEK_NUM']   = df['WEEK_NUM'].astype('int')
df['PTS_WINNER'] = df['PTS_WINNER'].astype('int')
df['PTS_LOSER']  = df['PTS_LOSER'].astype('int')
df['PTS_LOSER']  = df['YARDS_WINNER'].astype('int')
df['PTS_LOSER']  = df['TURNOVERS_WINNER'].astype('int')
df['PTS_LOSER']  = df['YARDS_LOSER'].astype('int')
df['PTS_LOSER']  = df['TURNOVERS_LOSER'].astype('int')

## Create a column that denotes a win or a tie (could be used for prediction measuring)
df.loc[df['PTS_WINNER']>df['PTS_LOSER'],'WIN_OR_TIE'] = 1
df['WIN_OR_TIE'].fillna(0,inplace=True)

df.to_csv('../../data/cleaned/NFL_wk_by_wk_cleaned.csv', index=False)

df.tail()

,WEEK_NUM,GAME_DAY_OF_WEEK,GAME_DATE,GAMETIME,WINNER,GAME_LOCATION,LOSER,BOXSCORE_LINK,PTS_WINNER,PTS_LOSER,YARDS_WINNER,TURNOVERS_WINNER,YARDS_LOSER,TURNOVERS_LOSER,YEAR,WIN_OR_TIE
8907,20,Sun,2023-01-22,3:00PM,Cincinnati Bengals,@,Buffalo Bills,/boxscores/202301220buf.htm,27,1,412,0,325,1,22-23,1.0
8908,20,Sun,2023-01-22,6:30PM,San Francisco 49ers,NULL_VALUE,Dallas Cowboys,/boxscores/202301220sfo.htm,19,2,312,1,282,2,22-23,1.0
8909,21,Sun,2023-01-29,3:00PM,Philadelphia Eagles,NULL_VALUE,San Francisco 49ers,/boxscores/202301290phi.htm,31,3,269,0,164,3,22-23,1.0
8910,21,Sun,2023-01-29,6:30PM,Kansas City Chiefs,NULL_VALUE,Cincinnati Bengals,/boxscores/202301290kan.htm,23,2,357,1,309,2,22-23,1.0
8911,22,Sun,2023-02-12,6:30PM,Kansas City Chiefs,N,Philadelphia Eagles,/boxscores/202302120phi.htm,38,1,340,0,417,1,22-23,1.0


In [3]:
## Doing some sanity checking
print(f"Unique seasons: {df['YEAR'].unique()}")
print(f"Unique values for Week_num: {df['WEEK_NUM'].unique()}") # 1-22
print(f"Number of Unique Winning & Losing teams: {len(df['WINNER'].unique())} | {len(df['LOSER'].unique())}") #32

Unique seasons: ['91-92' '92-93' '93-94' '94-95' '95-96' '96-97' '97-98' '98-99' '99-00'
 '00-01' '01-02' '02-03' '03-04' '04-05' '05-06' '06-07' '07-08' '08-09'
 '09-10' '10-11' '11-12' '12-13' '13-14' '14-15' '15-16' '16-17' '17-18'
 '18-19' '19-20' '20-21' '21-22' '22-23']
Unique values for Week_num: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]
Number of Unique Winning & Losing teams: 32 | 32


# Establishing ranking table

In [21]:
## Pull out all NFL teams
NFL_TEAMS = sorted(df['WINNER'].unique())

## Pull out Years and Week numbers
NFL_YEARS = sorted(df['YEAR'].unique())
NFL_WEEKS = sorted(df['WEEK_NUM'].unique())

dataframe_builder_list = []
for team in NFL_TEAMS:
    for year in NFL_YEARS:
        for week in NFL_WEEKS:
            dataframe_builder_list.append( (team, year, week) )

df_team_elo = pd.DataFrame.from_records(dataframe_builder_list, columns=['NFL_TEAM', 'NFL_YEAR', 'NFL_WEEK']).drop_duplicates()
df_team_elo['ELO'] = np.nan
df_team_elo.loc[(df_team_elo['NFL_YEAR']==sorted_years[0])&(df_team_elo['NFL_WEEK']==1),'ELO'] = 1300

## Make dataframe year into categories
df_team_elo['NFL_YEAR'] = pd.Categorical(df_team_elo['NFL_YEAR'], sorted_years)

## Remove week 22 from seasons prior to '21-22'
df_team_elo = df_team_elo.loc[~((df_team_elo['NFL_YEAR'].isin(sorted_years[:increase_in_number_of_weeks_in_season]))&
                                (df_team_elo['NFL_WEEK']==22))]

## Sort Dataframe
df_team_elo = df_team_elo.sort_values(['NFL_YEAR','NFL_WEEK','NFL_TEAM'], ignore_index=True)

## Remove rows where teams didn't 'exist' yet
#   Ex: Carolina Panthers didn't start until the 1995 season, remove all years prior

## Carolina Panthers
carolina_panthers_start_year = '95-96'
carolina_panthers_year_list_to_remove = df_team_elo[(df_team_elo['NFL_TEAM']=='Carolina Panthers')&
                                                    (df_team_elo['NFL_YEAR'].isin(sorted_years[:sorted_years.index(carolina_panthers_start_year)]))].index.values

## Jacksonville Jaguars
jacksonville_jaguars_start_year = '95-96'
jacksonville_jaguars_year_list_to_remove = df_team_elo[(df_team_elo['NFL_TEAM']=='Jacksonville Jaguars')&
                                                       (df_team_elo['NFL_YEAR'].isin(sorted_years[:sorted_years.index(jacksonville_jaguars_start_year)]))].index.values

## Baltimore Ravens
baltimore_ravens_start_year = '96-97'
baltimore_ravens_year_list_to_remove = df_team_elo[(df_team_elo['NFL_TEAM']=='Baltimore Ravens')&
                                                   (df_team_elo['NFL_YEAR'].isin(sorted_years[:sorted_years.index(baltimore_ravens_start_year)]))].index.values

## Houston Texans
houston_texans_start_year = '02-03'
houston_texans_year_list_to_remove = df_team_elo[(df_team_elo['NFL_TEAM']=='Houston Texans')&
                                                 (df_team_elo['NFL_YEAR'].isin(sorted_years[:sorted_years.index(houston_texans_start_year)]))].index.values

idx_to_remove = np.concatenate( (carolina_panthers_year_list_to_remove,
                                jacksonville_jaguars_year_list_to_remove,
                                baltimore_ravens_year_list_to_remove, 
                                houston_texans_year_list_to_remove), axis=None)

df_team_elo.drop(idx_to_remove, inplace=True)


df_team_elo.head()

,NFL_TEAM,NFL_YEAR,NFL_WEEK,ELO
0,Arizona Cardinals,91-92,1,1300.0
1,Atlanta Falcons,91-92,1,1300.0
3,Buffalo Bills,91-92,1,1300.0
5,Chicago Bears,91-92,1,1300.0
6,Cincinnati Bengals,91-92,1,1300.0


In [5]:
def elo_win_probability(rating_teamA, rating_teamB):
    ## Probability of of winning of rating_teamA
    P1 = 1.0 / (1.0 + 10**((rating_teamB-rating_teamA)/400))

    ## Probability of of winning of rating_teamB
    P2 = 1.0 / (1.0 + 10**((rating_teamA-rating_teamB)/400))
    
    return(P1, P2)

def update_elo(winning_team_elo, losing_team_elo, K=20 ):
    winners_chances, losers_chances = elo_win_probability(winning_team_elo, losing_team_elo)
    
    new_winner_elo = winning_team_elo + K * (1 - winners_chances)
    new_loser_elo  = losing_team_elo  + K * (0 - losers_chances )
    
    return(new_winner_elo, new_loser_elo)

def evaluate_ranking_system(df, df_team, ranking_system=elo_win_probability):

    temp = df.loc[:,['WEEK_NUM','WINNER','LOSER','YEAR','WIN_OR_TIE']].copy()
    elo_prob = []

    for i in temp.iterrows():

        winning_team_name = i[1]['WINNER']
        losing_team_name  = i[1]['LOSER']

        YEAR = i[1]['YEAR']
        WEEK = i[1]['WEEK_NUM']

        elo_prob.append(ranking_system(df_team_elo.loc[ (df_team_elo['NFL_TEAM'] == winning_team_name)&
                                                        (df_team_elo['NFL_YEAR'] == YEAR)&
                                                        (df_team_elo['NFL_WEEK'] == WEEK),'ELO'].values[0],
                                       df_team_elo.loc[ (df_team_elo['NFL_TEAM'] == losing_team_name)&
                                                        (df_team_elo['NFL_YEAR'] == YEAR)&
                                                        (df_team_elo['NFL_WEEK'] == WEEK),'ELO'].values[0])[0] )

    temp['ELO_PROB'] = elo_prob

    temp.loc[(temp['WIN_OR_TIE']==1)&(temp['ELO_PROB']>.5),'CORRECT'] = 1
    temp['CORRECT'].fillna(0,inplace=True)

    for i in temp['YEAR'].unique():
        print(f"{i}: {(temp.loc[temp['YEAR']==i,'CORRECT'].sum()/temp.loc[temp['YEAR']==i,:].shape[0]):.0%} correct on the season")

    print(f"Overall: {(temp['CORRECT'].sum()/temp.shape[0]):.0%}")

    elo_prob = None
    del elo_prob

In [6]:
## Create a new column of Elo per week of a season
# for k in range(20,31):
k=20
for idx, yr in enumerate(tqdm(sorted_years)):
    curr_year = yr
    if curr_year != sorted_years[-1]:
        next_year = sorted_years[idx+1]
    else: 
        break

    number_of_weeks_in_curr_year = len(df.loc[df['YEAR'] == curr_year, 'WEEK_NUM'].unique())

    ## pull out all team names this season, we want to create a list to check against when a new team joins the NFL
    teams_this_season = df_team_elo.loc[(df_team_elo['NFL_YEAR'] == curr_year),'NFL_TEAM'].unique()

    for wk in range(1,number_of_weeks_in_curr_year+1):
        for i in df.loc[((df['WEEK_NUM']==wk)&(df['YEAR']==curr_year)),['WINNER','LOSER']].iterrows():

            winning_team_name = i[1]['WINNER']
            losing_team_name  = i[1]['LOSER']
            
            ## Calculate the winner's and loser's new Elo ranking.
            winner_elo, loser_elo = update_elo( df_team_elo.loc[(df_team_elo['NFL_TEAM'] == winning_team_name )&
                                                                (df_team_elo['NFL_YEAR']  == curr_year        )&
                                                                (df_team_elo['NFL_WEEK']  == wk               ),'ELO'].values[0],
                                                df_team_elo.loc[(df_team_elo['NFL_TEAM'] == losing_team_name  )&
                                                                (df_team_elo['NFL_YEAR']  == curr_year        )&
                                                                (df_team_elo['NFL_WEEK']  == wk               ),'ELO'].values[0],
                                                K=k)

            ## Update Elo for next seasons week 1
            if wk == number_of_weeks_in_curr_year:
                ## Update winning team's ELO
                df_team_elo.loc[(df_team_elo['NFL_TEAM'] == winning_team_name)&
                                (df_team_elo['NFL_YEAR'] == next_year        )&
                                (df_team_elo['NFL_WEEK'] == 1               ),'ELO'] = winner_elo

                ## Update losing team's ELO
                df_team_elo.loc[(df_team_elo['NFL_TEAM'] == losing_team_name )&
                                (df_team_elo['NFL_YEAR'] == next_year        )&
                                (df_team_elo['NFL_WEEK'] == 1               ),'ELO'] = loser_elo

            ## Update Elo for next week in season
            else:

                ## Update winning team's ELO
                df_team_elo.loc[(df_team_elo['NFL_TEAM'] == winning_team_name)&
                                (df_team_elo['NFL_YEAR'] == curr_year        )&
                                (df_team_elo['NFL_WEEK'] == wk+1             ),'ELO'] = winner_elo

                ## Update losing team's ELO
                df_team_elo.loc[(df_team_elo['NFL_TEAM'] == losing_team_name )&
                                (df_team_elo['NFL_YEAR'] == curr_year        )&
                                (df_team_elo['NFL_WEEK'] == wk+1             ),'ELO'] = loser_elo

                

        ## Set week 1's ELO of next year equal to the ELO of the last week of the current year
        if wk == number_of_weeks_in_curr_year:
            ## Get boolean values for next year, week 1, and ELO is null
            null_idx = df_team_elo.loc[(df_team_elo['NFL_YEAR'] == next_year)&(df_team_elo['NFL_WEEK'] == 1),'ELO'].isnull()
            
            ## Take list of unique team names that appear in the above null_idx list
            teams_with_no_game_this_week = df_team_elo.loc[(df_team_elo['NFL_YEAR'] == next_year)&(df_team_elo['NFL_WEEK'] == 1),:].loc[null_idx,'NFL_TEAM'].unique()

            ## For each team that has a null value for their ELO ranking for Next year's week 1
            for team in teams_with_no_game_this_week:
                
                ## If the team has just joined the NFL, set its' ELO to 1300 (starting_value)
                if team not in teams_this_season:
                    df_team_elo.loc[(df_team_elo['NFL_TEAM'] == team     )&
                                    (df_team_elo['NFL_YEAR'] == next_year)&
                                    (df_team_elo['NFL_WEEK'] == 1        ),'ELO'] = 1300
                
                else:
                    
                    ## Set the ELO value of next year's week 1 (the null value) to the value of the last week of last season (should be a float, not null)
                    df_team_elo.loc[(df_team_elo['NFL_TEAM'] == team     )&
                                    (df_team_elo['NFL_YEAR'] == next_year)&
                                    (df_team_elo['NFL_WEEK'] == 1        ),'ELO'] = df_team_elo.loc[(df_team_elo['NFL_TEAM'] == team                        )&
                                                                                                    (df_team_elo['NFL_YEAR'] == curr_year                   )&
                                                                                                    (df_team_elo['NFL_WEEK'] == number_of_weeks_in_curr_year),'ELO'].values[0]


        ## This catches bye weeks and not making the playoffs  (so they would not have a game to iterate over in the winner/loser elo calculation) 
        else:
            ## Get boolean values for current year, next week (wk+1), and ELO is null
            null_idx = df_team_elo.loc[(df_team_elo['NFL_YEAR'] == curr_year)&(df_team_elo['NFL_WEEK'] == wk+1),'ELO'].isnull()

            ## Take list of unique team names that appear in the above null_idx list
            teams_with_no_game_this_week = df_team_elo.loc[(df_team_elo['NFL_YEAR'] == curr_year)&(df_team_elo['NFL_WEEK'] == wk+1),:].loc[null_idx,'NFL_TEAM'].unique()
            
            ## For each team that has a null value for their ELO ranking for the next week
            for team in teams_with_no_game_this_week:
                
                ## Set the ELO value of next week (the null value) to the value of last week (should be a float, not null)
                df_team_elo.loc[(df_team_elo['NFL_TEAM'] == team        )&
                                (df_team_elo['NFL_YEAR'] == curr_year   )&
                                (df_team_elo['NFL_WEEK'].isin([wk,wk+1])),'ELO'] = df_team_elo.loc[(df_team_elo['NFL_TEAM'] == team        )&
                                                                                                    (df_team_elo['NFL_YEAR'] == curr_year   )&
                                                                                                    (df_team_elo['NFL_WEEK'].isin([wk,wk+1])),'ELO'].fillna(method='ffill')

    # Offseason decay -- regress ELO's to the mean
    curr_season_mean_elo = df_team_elo.loc[(df_team_elo['NFL_YEAR'] == next_year)&
                                            (df_team_elo['NFL_WEEK'] == 1 )&
                                            (df_team_elo['NFL_TEAM'].isin(teams_this_season)),'ELO'].mean()
    df_team_elo.loc[(df_team_elo['NFL_YEAR'] == next_year)&
                    (df_team_elo['NFL_WEEK'] == 1 )&
                    (df_team_elo['NFL_TEAM'].isin(teams_this_season)),'ELO'] =  curr_season_mean_elo*(1/3.0) + \
                                                                                df_team_elo.loc[(df_team_elo['NFL_YEAR'] == next_year)&
                                                                                                (df_team_elo['NFL_WEEK'] == 1 )&
                                                                                                (df_team_elo['NFL_TEAM'].isin(teams_this_season)),'ELO']\
                                                                                *(1-(1/3.0))

 97%|███████████████████████████████████████████████████████████████████████████████▌  | 32/33 [00:51<00:01,  1.61s/it]


In [11]:
df_team_elo.to_csv('data/cleaned/NFL_Team_Elo.csv', index=False)

In [7]:
## Average ELO by team
df_team_elo.loc[:,['NFL_TEAM','ELO']].groupby(['NFL_TEAM']).mean().sort_values('ELO', ascending=False)

,ELO
NFL_TEAM,
New England Patriots,1438.855043
Pittsburgh Steelers,1398.077679
Green Bay Packers,1397.537720
Denver Broncos,1359.266002
Kansas City Chiefs,1358.708756
Dallas Cowboys,1351.378627
Philadelphia Eagles,1345.751068
Baltimore Ravens,1345.101050
Indianapolis Colts,1339.315461


In [8]:
## Most recent ELO rating
df_team_elo.loc[(df_team_elo['NFL_YEAR']=='22-23')&(df_team_elo['NFL_WEEK']==22),:].sort_values(['ELO'], ascending=False)

,NFL_TEAM,NFL_YEAR,NFL_WEEK,ELO
21551,Kansas City Chiefs,22-23,22,1615.157713
21542,Cincinnati Bengals,22-23,22,1556.339338
21539,Buffalo Bills,22-23,22,1538.485800
21563,San Francisco 49ers,22-23,22,1536.909774
21561,Philadelphia Eagles,22-23,22,1487.961809
21544,Dallas Cowboys,22-23,22,1413.839583
21562,Pittsburgh Steelers,22-23,22,1406.260527
21556,Minnesota Vikings,22-23,22,1380.636716
21547,Green Bay Packers,22-23,22,1355.311810
21557,New England Patriots,22-23,22,1312.247292


In [9]:
evaluate_ranking_system(df, df_team_elo)

91-92: 59% correct on the season
92-93: 68% correct on the season
93-94: 62% correct on the season
94-95: 60% correct on the season
95-96: 54% correct on the season
96-97: 62% correct on the season
97-98: 61% correct on the season
98-99: 61% correct on the season
99-00: 57% correct on the season
00-01: 63% correct on the season
01-02: 62% correct on the season
02-03: 57% correct on the season
03-04: 64% correct on the season
04-05: 60% correct on the season
05-06: 61% correct on the season
06-07: 60% correct on the season
07-08: 64% correct on the season
08-09: 64% correct on the season
09-10: 64% correct on the season
10-11: 54% correct on the season
11-12: 61% correct on the season
12-13: 62% correct on the season
13-14: 59% correct on the season
14-15: 66% correct on the season
15-16: 64% correct on the season
16-17: 60% correct on the season
17-18: 61% correct on the season
18-19: 60% correct on the season
19-20: 61% correct on the season
20-21: 66% correct on the season
21-22: 64%